In [ ]:
import finnhub
import pandas as pd

finnhub_client = finnhub.Client(
    api_key="cujft39r01qm7p9o29jgcujft39r01qm7p9o29k0")

value = finnhub_client.stock_symbols('US')
df = pd.DataFrame(value)
print(df["symbol"])

In [42]:
series = df["symbol"]
search_term = "BINANCE:BTCUSDT"
# search_term = "GOOGL"
search_terms = ['GOOGL', 'AMZN', 'AAPL', 'T', 'DELL', 'HPQ', 'INTC', 'META', 'MSFT', 'NVDA']
case_insensitive_matches = series[series.str.contains(search_term, case=False, na=False)]
# case_insensitive_matches = series[series.isin(search_terms)]
print(case_insensitive_matches)

Series([], Name: symbol, dtype: object)


In [43]:
print(finnhub_client.symbol_lookup("BINANCE:BTCUSDT"))

{'count': 1, 'result': [{'description': 'Binance BTC/USDT', 'displaySymbol': 'BTC/USDT', 'symbol': 'BINANCE:BTCUSDT', 'type': 'Crypto'}]}


In [ ]:
value = finnhub_client.crypto_symbols('BINANCE')
print(value)

In [65]:
df = pd.DataFrame(value)
search_term = 'BTCUSD'
result = df["symbol"][df["symbol"].str.contains(search_term, case=False, na=False)]
print(result.to_list())

['BINANCE:BTCUSDS', 'BINANCE:BTCUSDP', 'BINANCE:WBTCUSDT', 'BINANCE:BTCUSDT', 'BINANCE:BTCUSDC']


In [ ]:
#https://pypi.org/project/websocket_client/
import json
import websocket
import traceback


class Test:
  def __init__(self):
     self.message_count = 0

  def set_tickers(self, tickers):
     self.tickers = tickers

  def append_to_file(filepath, text):
    """Appends text to a file. Creates the file if it doesn't exist."""
    try:
        with open(filepath, 'a') as f:  # 'a' mode for appending
            f.write(text)
    except Exception as e:
        print(f"An error occurred: {e}")

  def on_message(self, ws, message):
      # data = json.load(message)
      data = dict(json.loads(message))
      # if(data && data)
      if("data" in data.keys()):
         print(data["data"][0]["s"])
      print("*"*20)
      print(f"Message: {data}")
      print("*"*20)
      # print(data)
      # self.message_count += 1
      ws.close()
      # if(self.message_count >= 1):
      # print("----------------------------------------------------")


  def on_error(self, ws, error):
      traceback.print_exc()
      raise error

  def on_close(self, ws, close_status_code, close_msg):
      traceback.print_exc()
      print("### closed ###")

  def on_open(self, ws):
     for ticker in self.tickers:
        ws.send(f'{{"type":"subscribe", "symbol":"{ticker}"}}')

  def run_websocket(self, tickers):
    self.message_count = 0
    self.set_tickers(tickers)
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("wss://ws.finnhub.io?token=cujft39r01qm7p9o29jgcujft39r01qm7p9o29k0",
                              on_message = self.on_message,
                              on_error = self.on_error,
                              on_close = self.on_close)
    ws.on_open = self.on_open
    ws.run_forever()

if __name__ == "__main__":
  test = Test()
  tickers = ['BINANCE:BTCUSDT', 'BINANCE:ETHUSDC', 'BINANCE:ETHUSDP', 'BINANCE:ETHUSDT', 'BINANCE:BTCUSDC']
  test.run_websocket(tickers)


In [140]:
import re
from confluent_kafka import Consumer, KafkaError, DeserializingConsumer
from confluent_kafka.serialization import StringDeserializer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer


import FinnhubProducer.utils.settings as UTILS
from FinnhubConsumer.utils.utilities import Utilities

# Kafka configuration
bootstrap_servers = UTILS.BOOTSTRAP_SERVERS # Replace with your Kafka brokers
schema_registry_url = UTILS.SCHEMA_REGISTRY_URL # Replace with your Schema Registry URL
# topic_names = ["LATEST_PRICES"]  # Replace with the name of your KSQL stream
topic_names = ["STOCK_SUMMARY", "LATEST_PRICES"]  # Replace with the name of your KSQL stream
group_id = "malmike.finnhub.avro.consumer.1"  # Replace with a unique consumer group ID

# Schema Registry configuration
schema_registry_conf = {'url': schema_registry_url}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Avro deserializer
avro_deserializer = AvroDeserializer(schema_registry_client)  # reader_schema can be None

# Consumer configuration
consumer_conf = {
    'bootstrap.servers': bootstrap_servers,
    'key.deserializer': avro_deserializer,  # Assuming your keys are strings
    'value.deserializer': avro_deserializer,
    'group.id': group_id,
    'auto.offset.reset': 'earliest'  # Or 'latest' depending on your needs
}

# Create the deserializing consumer
consumer = DeserializingConsumer(consumer_conf)

def consume_from_kafka(topic_names):
    # Subscribe to the topic
    consumer.subscribe(topic_names)
    try:
        while True:
            msg = consumer.poll(1.0)  # Adjust timeout as needed

            if msg is None:
                continue

            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    print('End of partition reached')
                    continue
                else:
                    print(f"Error: {msg.error()}")
                    break
            topic = msg.topic()
            key = msg.key()
            value = msg.value()

            if value is not None:  # Handle potential null values
                # print(f"Key: {key}, Value: {value}")
                yield(topic, key, value)
                # Process the Avro record (value) here
            else:
                yield(topic,key, None)
                print(f"Key: {key}, Value: None")  # Handle null values if needed

    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

def remove_no_printable_characters(input_str: str) -> str:
    return re.sub(r'[\x00-\x1F]', "", input_str)

def compute_total(missing_symbols: list[str], latest_prices: dict[str, float], total: float):
    for symbol in missing_symbols:
        total += latest_prices[symbol]
    return total

latest_prices = {}
for topic, key, value in consume_from_kafka(topic_names):
    if(topic == "LATEST_PRICES"):
        latest_prices[key] = value["LAST_PRICE"]

    if(topic == "STOCK_SUMMARY"):
        if value is not None:
            symbols = {remove_no_printable_characters(item) for item in value["SYMBOLS"]}
            missing_symbols = list(latest_prices.keys() - symbols)
            total = compute_total(
                missing_symbols=missing_symbols,
                latest_prices=latest_prices,
                total=value["TOTAL_PRICE"]
            )
            all_symbols = latest_prices.keys() if len(latest_prices) == len(missing_symbols)+len(symbols) else list(set(missing_symbols) | symbols)
            print("-"*30)
            print(f"""Topic: {topic}, Key: {key}, Total Price:{total}, All Symbols: {all_symbols}
            """)
            print("-"*30)

#             with open("my_file.txt", "a") as file:
#                 data = f"""
# ---------------------------------------------------------
# Topic: {topic}, Key: {key},
# Value:{value["TOTAL_PRICE"]},
# Symbols:{symbols},
# Missing: {missing_symbols},
# All Symbols: {list(latest_prices.keys())}
# ---------------------------------------------------------
#                 """
#                 file.write(data)

#             print("-"*30)
#             print(f"""
# Topic: {topic}, Key: {key},
# Value:{value["TOTAL_PRICE"]},
# Symbols:{symbols},
# Missing: {missing_symbols},
# All Symbols: {list(latest_prices.keys())}
#             """)
#             print("-"*30)




------------------------------
Topic: STOCK_SUMMARY, Key: 1739279652210, Total Price:201048.00166666668, All Symbols: ['BINANCE:BTCUSDT', 'BINANCE:ETHUSDC', 'BINANCE:BTCUSDC', 'BINANCE:ETHUSDT']
            
------------------------------
------------------------------
Topic: STOCK_SUMMARY, Key: 1739279653154, Total Price:198355.08250000002, All Symbols: dict_keys(['BINANCE:BTCUSDT', 'BINANCE:BTCUSDC', 'BINANCE:ETHUSDT'])
            
------------------------------
------------------------------
Topic: STOCK_SUMMARY, Key: 1739279654151, Total Price:198355.9525, All Symbols: dict_keys(['BINANCE:BTCUSDT', 'BINANCE:BTCUSDC', 'BINANCE:ETHUSDT'])
            
------------------------------
------------------------------
Topic: STOCK_SUMMARY, Key: 1739279655157, Total Price:198355.08000000002, All Symbols: dict_keys(['BINANCE:BTCUSDT', 'BINANCE:BTCUSDC', 'BINANCE:ETHUSDT'])
            
------------------------------
------------------------------
Topic: STOCK_SUMMARY, Key: 1739279801387, To